In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Dantte Roberto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import unicodedata

## Extração dos dados

In [3]:


#Criando um dicionário com todos os caminhos dos arquivos. Isso vai ser bom porque utilizarei as chaves para criar uma coluna de "Ano"
arquivos = {
    '2022': 'data/156_2022.csv',#utilizamos a base de 2022 pois a de 2023 estava dando erro na hora de juntar as tabelas, todas as colunas estavam ficando juntas em apenas umna
    '2024': 'data/156_2024.csv',
    '2025': 'data/156_2025.csv'
}

lista_dataframes = [] #Para armazenar todos os dataframes

for ano, arquivo in arquivos.items():
    try:
        if ano == '2022':
            df = pd.read_csv(arquivo,sep=';',encoding='utf-8') # O arquivo do ano de 2022 esta em "UTF-8" se se usassemos "latin1" os caracteres "Ç" "~" iriam ficar errados
        else:
            df = pd.read_csv(arquivo,sep=';',encoding='latin1')
        
        df['_id'] = range(1, len(df) + 1) #Criação do campo id pois os arquivos csv vieram sem

        cols = ['_id'] + [c for c in df.columns if c != '_id']
        df = df[cols]
        
        df['ano_origem'] = ano #criando uma nova coluna para salvar o ano dos chamados

        lista_dataframes.append(df)
    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")


df = pd.concat(lista_dataframes, ignore_index=True)


## Transformação dos Dados


In [4]:
#Verificando o resultado final e as informações sobre os tipos de dados
display(df.head(5))
display(df.info())

,_id,GRUPOSERVICO_CODIGO,GRUPOSERVICO_DESCRICAO,SERVICO_CODIGO,SERVICO_DESCRICAO,LOGRADOURO,NUMERO,BAIRRO,RPA,DATA_DEMANDA,SITUACAO,DATA_ULT_SITUACAO,latitude,longitude,ano_origem
0,1,10,DENUNCIAS,700,DENUNCIA ARBOVIROSE ...,10t prazeres,,AREIAS,5,2022-06-20,PREPARAÇÃO,2022-09-22,-8.0981629,-34.9249594,2022
1,2,7,ARBORIZAÇÃO,16,VISTORIA DE ARVORES ...,1sb corrego do leoncio,,NOVA DESCOBERTA,3,2022-07-27,ATENDIDA,2022-09-08,-8.0028869,-34.9274109,2022
2,3,7,ARBORIZAÇÃO,16,VISTORIA DE ARVORES ...,1sb corrego do leoncio,,NOVA DESCOBERTA,3,2022-10-16,PREPARAÇÃO,2022-10-17,-8.0028869,-34.9274109,2022
3,4,12,ESCADARIA,57,RECUP. DE ESCADARIA ...,1sb corrego do leoncio,,NOVA DESCOBERTA,3,2022-02-16,CADASTRADA,2022-02-16,-8.0028869,-34.9274109,2022
4,5,12,ESCADARIA,57,RECUP. DE ESCADARIA ...,1sb do sargento,,NOVA DESCOBERTA,3,2022-06-02,CADASTRADA,2022-06-02,,,2022


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291681 entries, 0 to 291680
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   _id                     291681 non-null  int64 
 1   GRUPOSERVICO_CODIGO     291681 non-null  int64 
 2   GRUPOSERVICO_DESCRICAO  291681 non-null  object
 3   SERVICO_CODIGO          291681 non-null  int64 
 4   SERVICO_DESCRICAO       291681 non-null  object
 5   LOGRADOURO              291681 non-null  object
 6   NUMERO                  291681 non-null  object
 7   BAIRRO                  291681 non-null  object
 8   RPA                     291681 non-null  int64 
 9   DATA_DEMANDA            291681 non-null  object
 10  SITUACAO                291681 non-null  object
 11  DATA_ULT_SITUACAO       291681 non-null  object
 12  latitude                140737 non-null  object
 13  longitude               140737 non-null  object
 14  ano_origem              291681 non-n

None

### Padronizando a Escritas das Tabelas

- Em algumas tabelas alguns nomes apresentavam acentos e 'ç' ja em outras não por isso optamos por alterar a tabela de 2022 que continha esses caracteres e substituilos.
- Algumas colunas vieram preenchidas com letras minuscula

In [5]:
colunas_de_texto = df.select_dtypes(include=['object']).columns
for col in colunas_de_texto:
    df[col] = df[col].apply(
        lambda x: unicodedata.normalize('NFKD', str(x)).encode('ascii', 'ignore').decode('utf-8').strip() 
        if pd.notna(x) else x)
    df[col] = df[col].str.upper()
    

In [6]:
#Verificando o resultado final e as informações sobre os tipos de dados
display(df.head(5))
display(df.info())

,_id,GRUPOSERVICO_CODIGO,GRUPOSERVICO_DESCRICAO,SERVICO_CODIGO,SERVICO_DESCRICAO,LOGRADOURO,NUMERO,BAIRRO,RPA,DATA_DEMANDA,SITUACAO,DATA_ULT_SITUACAO,latitude,longitude,ano_origem
0,1,10,DENUNCIAS,700,DENUNCIA ARBOVIROSE,10T PRAZERES,,AREIAS,5,2022-06-20,PREPARACAO,2022-09-22,-8.0981629,-34.9249594,2022
1,2,7,ARBORIZACAO,16,VISTORIA DE ARVORES,1SB CORREGO DO LEONCIO,,NOVA DESCOBERTA,3,2022-07-27,ATENDIDA,2022-09-08,-8.0028869,-34.9274109,2022
2,3,7,ARBORIZACAO,16,VISTORIA DE ARVORES,1SB CORREGO DO LEONCIO,,NOVA DESCOBERTA,3,2022-10-16,PREPARACAO,2022-10-17,-8.0028869,-34.9274109,2022
3,4,12,ESCADARIA,57,RECUP. DE ESCADARIA,1SB CORREGO DO LEONCIO,,NOVA DESCOBERTA,3,2022-02-16,CADASTRADA,2022-02-16,-8.0028869,-34.9274109,2022
4,5,12,ESCADARIA,57,RECUP. DE ESCADARIA,1SB DO SARGENTO,,NOVA DESCOBERTA,3,2022-06-02,CADASTRADA,2022-06-02,,,2022


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291681 entries, 0 to 291680
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   _id                     291681 non-null  int64 
 1   GRUPOSERVICO_CODIGO     291681 non-null  int64 
 2   GRUPOSERVICO_DESCRICAO  291681 non-null  object
 3   SERVICO_CODIGO          291681 non-null  int64 
 4   SERVICO_DESCRICAO       291681 non-null  object
 5   LOGRADOURO              291681 non-null  object
 6   NUMERO                  291681 non-null  object
 7   BAIRRO                  291681 non-null  object
 8   RPA                     291681 non-null  int64 
 9   DATA_DEMANDA            291681 non-null  object
 10  SITUACAO                291681 non-null  object
 11  DATA_ULT_SITUACAO       291681 non-null  object
 12  latitude                140737 non-null  object
 13  longitude               140737 non-null  object
 14  ano_origem              291681 non-n

None

### Verificando valores únicos nas colunas

In [7]:
df.nunique(dropna=False)

_id                       108082
GRUPOSERVICO_CODIGO           33
GRUPOSERVICO_DESCRICAO        26
SERVICO_CODIGO               176
SERVICO_DESCRICAO            129
LOGRADOURO                 11209
NUMERO                      8789
BAIRRO                        99
RPA                            6
DATA_DEMANDA                3469
SITUACAO                       7
DATA_ULT_SITUACAO           1007
latitude                   68175
longitude                  64337
ano_origem                     3
dtype: int64

### Verificando a quantidade de Valores vazios por coluna

In [8]:
# Contando strings vazias nas colunas object
string_vazia_count = (df.select_dtypes(include=['object']) == '').sum()
print("Contagem de Strings Vazias por Coluna (''):")
print(string_vazia_count)

Contagem de Strings Vazias por Coluna (''):
GRUPOSERVICO_DESCRICAO        0
SERVICO_DESCRICAO             0
LOGRADOURO                    0
NUMERO                    11197
BAIRRO                        0
DATA_DEMANDA                  0
SITUACAO                      0
DATA_ULT_SITUACAO             0
latitude                  19863
longitude                 19863
ano_origem                    0
dtype: int64


- Número: 11.197 valores ausentes. Isso é comum em dados de logradouros, onde nem todos os chamados possuem um número específico de porta.
- Latitude e Longitude contem a mesma quantidade de linhas vazias indicando que são chamados onde a localização geográfica não está disponível para registro.

#### Verificando os valores presentes na coluna "NUMERO"

In [9]:
print("Top 10 valores mais frequentes na coluna 'NUMERO':")
print(df['NUMERO'].value_counts().head(10))

Top 10 valores mais frequentes na coluna 'NUMERO':
NUMERO
SN     41037
00     27733
S/N    27666
       11197
0       4178
625     2597
000     1659
100     1301
23      1176
10      1117
Name: count, dtype: int64


#### Alterando os valores vazios de "numero", "latitute" e "longitude".

**Na coluna "NUMERO":** 
- valores como '0', '00', ... passamos para '0'.
- valores como '' e 'SN' transformamos em 'S/N'

**Na coluna "LATITUDE" e "LOGITUDE":**
- valores '' foram transformados em 'INDETERMINADO'
  

In [10]:
padronizacao_numero = {
    '': 'S/N',
    'SN': 'S/N',

    '0000': '0',
    '000':'0',
    '00': '0',
    '0':'0'
}
df['NUMERO'] = df['NUMERO'].replace(padronizacao_numero)


df['latitude'] = df['latitude'].replace('', 'INDETERMINADO')
df['longitude'] = df['longitude'].replace('', 'INDETERMINADO')

print("Valores únicos e contagem após preenchimento:")
print("\nNUMERO (Top 5):")
print(df['NUMERO'].value_counts().head())

print("\nlatitude (Top 5):")
print(df['latitude'].value_counts().head())

print("\nlongitude (Top 5):")
print(df['longitude'].value_counts().head())

Valores únicos e contagem após preenchimento:

NUMERO (Top 5):
NUMERO
S/N    79900
0      33895
625     2597
100     1301
23      1176
Name: count, dtype: int64

latitude (Top 5):
latitude
INDETERMINADO    19863
-8.0652743         260
-8.0531498         220
-8.0498212         208
-8.0279902         192
Name: count, dtype: int64

longitude (Top 5):
longitude
INDETERMINADO    19863
-34.8969194        260
-34.9459467        221
-34.8739471        208
-34.9029646        192
Name: count, dtype: int64


#### Criação da coluna "RESOLVIDO"

In [11]:
#Listando as possiveis situações 
print(df['SITUACAO'].unique())

['PREPARACAO' 'ATENDIDA' 'CADASTRADA' 'PENDENTE' 'EXECUCAO' 'FISCALIZACAO'
 'PENDENCIA']


In [12]:
#Criando a coluna 'RESOLVIDO' e adicionando o boleano true se a 'SITUACAO' for atendida
df['RESOLVIDO'] = df['SITUACAO'] == 'ATENDIDA'

In [13]:
print("Demonstração das colunas 'SITUACAO' e 'Resolvido':")
print(df[['SITUACAO', 'RESOLVIDO']].head(10))

Demonstração das colunas 'SITUACAO' e 'Resolvido':
     SITUACAO  RESOLVIDO
0  PREPARACAO      False
1    ATENDIDA       True
2  PREPARACAO      False
3  CADASTRADA      False
4  CADASTRADA      False
5  CADASTRADA      False
6  CADASTRADA      False
7  CADASTRADA      False
8    ATENDIDA       True
9  CADASTRADA      False


#### Criação da coluna "TEMPO_CONCLUSAO"

In [18]:
import numpy as np # Necessário para a condicional

In [19]:
# 1. Converter as colunas de texto para formato de data (datetime)
df['DATA_DEMANDA'] = pd.to_datetime(df['DATA_DEMANDA'], errors='coerce')
df['DATA_ULT_SITUACAO'] = pd.to_datetime(df['DATA_ULT_SITUACAO'], errors='coerce')

# 2. Calcular a diferença em dias
df['DIAS_DECORRIDOS'] = (df['DATA_ULT_SITUACAO'] - df['DATA_DEMANDA']).dt.days

# 3. Criar a coluna DURACAO com a lógica condicional
df['DURACAO'] = np.where(
    df['RESOLVIDO'] == True, 
    df['DIAS_DECORRIDOS'], 
    'EM ANDAMENTO'
)

In [20]:
# Visualizando o resultado
print(df[['DATA_DEMANDA', 'DATA_ULT_SITUACAO', 'RESOLVIDO', 'DURACAO']].head(10))

  DATA_DEMANDA DATA_ULT_SITUACAO  RESOLVIDO       DURACAO
0   2022-06-20        2022-09-22      False  EM ANDAMENTO
1   2022-07-27        2022-09-08       True            43
2   2022-10-16        2022-10-17      False  EM ANDAMENTO
3   2022-02-16        2022-02-16      False  EM ANDAMENTO
4   2022-06-02        2022-06-02      False  EM ANDAMENTO
5   2022-03-24        2022-03-24      False  EM ANDAMENTO
6   2022-02-24        2022-02-24      False  EM ANDAMENTO
7   2022-10-04        2022-10-04      False  EM ANDAMENTO
8   2022-04-04        2022-04-07       True             3
9   2022-07-20        2022-08-09      False  EM ANDAMENTO
